# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [2]:
data = get_date_list(data)
data = read_data(data)

Record num : 89426


In [3]:
for data['i'] in tqdm(range(0, data['df_len'])):
    
    data['df']['DateTime_frmt'][data['i']] = dt.datetime.strptime(data['df']['DateTime'][data['i']].split(".")[0],"%Y%m%d %H:%M:%S")
    
    data['ask'] = data['df']['Ask'][data['i']]
    data['bid'] = data['df']['Bid'][data['i']]  
    
    data['tick'] = (data['df']['Ask'][data['i']] + data['df']['Bid'][data['i']])/2
    data['df']['tick'][data['i']] = data['tick']
        
    # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------
    
        # sema before after loops --------------------------------
    if len(data['sema_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------


    # lema before after loops --------------------------------
    if len(data['lema_list']) < data['lema_len']:
        data = before_lema(data)        
        continue

    if len(data['lema_list']) == data['lema_len']:
        data = after_lema(data)                  
    # ----------------------------------------------------------
    
    data = get_ohlc(data)    
    
    data = get_position(data)
    if not data['position']:
        continue
        
    # Get Dirs --------------------------------
    if len(data['dir_list']) < 2:
        data['dir_list'].append(data['position'])   
        continue

    elif len(data['dir_list']) == 2:
        data = get_cross_dir(data)
    # ----------------------------------------------------------  
    
    data = get_trend_lines_back_line(data)
#     data = get_trend_lines_moving(data)
    data = trend_angle_check(data)

100%|███████████████████████████████████| 89426/89426 [13:33<00:00, 109.92it/s]


In [4]:
data['df']

,DateTime,Bid,Ask,tick,DateTime_frmt,sema,lema,position,direction,to_order,h_line_angle,l_line_angle,trend_angle,h_trend_calc_spot,l_trend_calc_spot,h,l
0,20210106 00:00:00.067,1.22985,1.22990,1.229875,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210106 00:00:00.169,1.22987,1.22992,1.229895,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210106 00:00:00.270,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210106 00:00:00.372,1.22989,1.22994,1.229915,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210106 00:00:00.980,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89421,20210106 14:16:19.743,1.23001,1.23003,1.230020,2021-01-06 14:16:19,1.229999,1.230319,down,,NaN,-24.051969,-20.926311,down,1.230838,1.229735,NaN,NaN
89422,20210106 14:16:19.794,1.23000,1.23001,1.230005,2021-01-06 14:16:19,1.229999,1.230319,down,,NaN,-24.051969,-20.926311,down,1.230837,1.229735,NaN,NaN
89423,20210106 14:16:21.624,1.22998,1.23000,1.229990,2021-01-06 14:16:21,1.229998,1.230318,down,,NaN,-24.051969,-20.926311,down,1.230837,1.229735,NaN,NaN
89424,20210106 14:16:22.233,1.22999,1.23002,1.230005,2021-01-06 14:16:22,1.229998,1.230318,down,,NaN,-24.051969,-20.926311,down,1.230837,1.229734,NaN,NaN


In [5]:
num_rec = len(data["df"])
hours = np.round((list(data['df']['DateTime_frmt'])[-1] - list(data['df']['DateTime_frmt'])[0]).total_seconds() / 60 / 60 , 2)
recsphour = int(num_rec/hours)

print(f'Records   : {num_rec}')
print(f'Hours     : {hours}')
print(f'Recs/hour : {recsphour}')
print(f'Recs/min  : {int(recsphour/60)}')
print(f'Recs/sec  : {np.round(recsphour/3600, 1)}')

data['df'].head()

Records   : 89426
Hours     : 14.27
Recs/hour : 6266
Recs/min  : 104
Recs/sec  : 1.7


,DateTime,Bid,Ask,tick,DateTime_frmt,sema,lema,position,direction,to_order,h_line_angle,l_line_angle,trend_angle,h_trend_calc_spot,l_trend_calc_spot,h,l
0,20210106 00:00:00.067,1.22985,1.22990,1.229875,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20210106 00:00:00.169,1.22987,1.22992,1.229895,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20210106 00:00:00.270,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20210106 00:00:00.372,1.22989,1.22994,1.229915,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20210106 00:00:00.980,1.22988,1.22993,1.229905,2021-01-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
plot_graph(data)

In [7]:
data['df'].to_csv('data/temp.csv')

In [8]:
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)